# Structured Logging Demo

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

In [ ]:
# import sqlite3
# from importlib import import_module
# from app.factories.logging_provider import LoggingProvider
# from app.extensions.context_providers.dummy_context_provider import DummyContextProvider
# from app.extensions.tool_providers.dummy_tool_provider import DummyToolProvider
# from app.extensions.agents.dummy_agent import DummyAgent
# from app.extensions.experiment_provider import ExperimentProvider

In [4]:
from app.utilities.db import init_db
import sqlite3
from app.factories.logging_provider import LoggingProvider
from app.extensions.context_providers import DummyContextProvider
from app.extensions.tool_providers import DummyToolProvider
from app.extensions.agents import DummyAgent
from app.extensions.experiment_provider import ExperimentProvider
from app.extensions.state_managers import DummyStateManager
from app.enums.system_enums import StateTransitionReason, SystemState

# Connect to the in-memory database
conn = sqlite3.connect(":memory:")

# Initialize all required logging tables explicitly
init_db(conn)

# Pass the initialized connection to your logger
logger = LoggingProvider(connection=conn)

# Instantiate other components
ctx = DummyContextProvider(logger=logger)
tool = DummyToolProvider(logger=logger)
agent = DummyAgent(logger=logger)

# Correctly instantiate DummyStateManager with required dependencies
state_manager = DummyStateManager(
    scoring_function=lambda context: 1.0,
    context_manager=ctx,
    agent=agent,
    logger=logger,
)

# Initialize Experiment
exp = ExperimentProvider(
    "demo",
    "demo experiment",
    "test",
    "v1",
    1,
    0.0,
    "engine",
    "eval",
    "1.0",
    logger=logger,
)

# Run dummy components explicitly
ctx.get_context(experiment_id="demo", round=0)
tool.run(experiment_id="demo", round=0)
agent.run(experiment_id="demo", round=0)

# Explicitly run state logic and transitions to test logging
state_manager.run(
    experiment_id="demo", system="dummy_system", state=SystemState.START, round=0
)
state_manager.transition_state(
    experiment_id="demo",
    round=0,
    from_state=SystemState.START,
    to_state=SystemState.GENERATE,
    reason=StateTransitionReason.AGENT_FAILURE,
)

exp.complete(1.0, True, "done")

In [6]:
cur = conn.cursor()
tables = [
    "context_retrieval_log",
    "tool_invocation_log",
    "agent_action_log",
    "system_event_log",
    "experiment_log",
    "state_log",
    "state_transition_log",
]
counts = {t: cur.execute(f"SELECT COUNT(*) FROM {t}").fetchone()[0] for t in tables}
print(counts)

# View state log results explicitly
state_logs = cur.execute("SELECT * FROM state_log").fetchall()
print("State Logs:", state_logs)

state_transition_logs = cur.execute("SELECT * FROM state_transition_log").fetchall()
print("State Transition Logs:", state_transition_logs)

{'context_retrieval_log': 2, 'tool_invocation_log': 1, 'agent_action_log': 3, 'system_event_log': 1, 'experiment_log': 1, 'state_log': 1, 'state_transition_log': 1}
State Logs: [('demo', 'dummy_system', 0, 'SystemState.START', 'run_state_logic', None, None, '2025-05-19T05:57:41.768460+00:00')]
State Transition Logs: [('demo', 0, 'SystemState.START', 'SystemState.GENERATE', 'StateTransitionReason.AGENT_FAILURE', '2025-05-19T05:57:41.769462+00:00')]
